## Summary 


1.  Set true weights and biases 
2.  Create synthetic data by adding normally distributed noise
3.  Load data into gluon data loader
4.  Create a model of Sequential type
5.  Add a dense layer with one node on the output 
6.  Initialize the weights of the model to normal weights
7.  Use L2Loss (Mean squared error) as the loss function
8.  Set the trainer to the model parameters with Stocastic gradient descent as the optimizing algorithm
9.  Set epochs and train your model 



In [1]:
!pip install mxnet-cu101mkl

     |████████████████████████████████| 587.7MB 31kB/s 
  Found existing installation: graphviz 0.10.1
    Uninstalling graphviz-0.10.1:
      Successfully uninstalled graphviz-0.10.1


In [0]:
import mxnet as mx
from mxnet import autograd, gluon
from mxnet.gluon import nn
mx.Context.default_ctx = mx.gpu(0)


In [0]:
  def synthetic_data(w, b, size):
    """
    weights: synethic_weight nd array

    bias: fixed value 

    size: the size of data set we want to create

    returns genearted data

    """

    # Genearte X with normal distribution

    X = mx.nd.normal(0,  0.1, (size, len(w)),
                     ctx = mx.gpu())
    
    
    y = mx.nd.dot(X, w) + b # add bias

    #adding additional noise which is normally distribued centered at 0 and variance : 0.1

    y = y + mx.nd.normal(0, 0.1, y.shape,
                         ctx = mx.gpu()) 

    return X,y

In [0]:
true_w = mx.nd.array([2, -3.4], ctx = mx.gpu())
true_b = 4.2
features, labels = synthetic_data(true_w, true_b, 1000)

In [0]:
def load_array(data_arrays, batch_size, is_train=True):
    """Construct a Gluon data loader"""
    dataset = gluon.data.ArrayDataset(*data_arrays)
    return gluon.data.DataLoader(dataset, batch_size, shuffle=is_train)

batch_size = 10
data_iter = load_array((features, labels), batch_size)

In [0]:
net = nn.Sequential()

adding output layer - no need to specify input layer

In [0]:
net.add(nn.Dense(1) )


Initialize model parameters with normal distributed weights mean = 0 and sigma = 0.01

In [0]:
from mxnet import init
net.initialize(init.Normal(sigma=0.01), ctx= mx.gpu())

L2 - mean squared eroor

In [0]:
from mxnet.gluon import loss as gloss
loss = gloss.L2Loss()  

In [0]:
trainer = gluon.Trainer(net.collect_params(), 'sgd', {'learning_rate': 0.03})

In [27]:
num_epochs = 3
for epoch in range(1, num_epochs + 1):
    for X, y in data_iter:
        with autograd.record():
            l = loss(net(X), y)
        l.backward()
        trainer.step(batch_size)
    l = loss(net(features), labels)
    print('epoch %d, loss: %f' % (epoch, l.mean().asnumpy()))

epoch 1, loss: 0.098309
epoch 2, loss: 0.074716
epoch 3, loss: 0.070594


Look up ones_like and autograd on the MXNet website.

ones_like: returns the ones in the same dimesions as input array
autograd: 

What are all the possible outputs after running np.random.choice(4, 2)?

In [39]:
mx.nd.random.randint(0, 4, (1,2))


[[2 3]]
<NDArray 1x2 @gpu(0)>